In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('/content/drive/MyDrive/data/손글씨/train/train_data.csv')

In [ ]:
train.head()

,file_name,label
0,train0001.png,8
1,train0002.png,8
2,train0003.png,8
3,train0004.png,8
4,train0005.png,8


In [ ]:
images = []
counts = 0

for image_name in train['file_name']:
    image = np.array(Image.open('/content/drive/MyDrive/data/손글씨/train/' + image_name)).flatten()
    images.append(image)
    counts+=1
    if counts%500==0:
      print(counts)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [ ]:
image_full = pd.DataFrame(images)
image_full['label'] = train['label']
image_full

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
4996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
4997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
4998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6


In [ ]:
from tensorflow.keras.utils import to_categorical

data_x = image_full.iloc[:,:-1]
data_x = data_x/255.0
data_x = data_x.values.reshape(-1,28,28,1)
data_y = to_categorical(train['label'], num_classes = 10)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data_x, data_y, test_size = .2, random_state = 1234, stratify = data_y)

In [ ]:
import tensorflow
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, input_shape=(28,28,1), padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tensorflow.keras.layers.MaxPool2D(pool_size=(2,2)),
    tensorflow.keras.layers.Dropout(0.2),

    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tensorflow.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
    tensorflow.keras.layers.MaxPool2D(pool_size=(2,2)),
    tensorflow.keras.layers.Dropout(0.2),

    tensorflow.keras.layers.Flatten(),
    tensorflow.keras.layers.Dense(units=256, activation='relu'),
    tensorflow.keras.layers.Dense(units=64, activation='relu'),
    tensorflow.keras.layers.Dropout(0.2),
    tensorflow.keras.layers.Dense(units = 10, activation='softmax')
])

optimizer = tensorflow.keras.optimizers.RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0 )
model.compile(optimizer = optimizer, 
              loss = "categorical_crossentropy", 
              metrics = ["accuracy"])
call_back = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.0001)

In [ ]:
history = model.fit(x_train, y_train, batch_size = 32, epochs = 30, verbose = 1)

Epoch 1/30
125/125 [==============================] - 15s 106ms/step - loss: 0.7073 - accuracy: 0.7492
Epoch 2/30
125/125 [==============================] - 18s 148ms/step - loss: 0.1606 - accuracy: 0.9492
Epoch 3/30
125/125 [==============================] - 13s 107ms/step - loss: 0.0998 - accuracy: 0.9710
Epoch 4/30
125/125 [==============================] - 13s 107ms/step - loss: 0.0755 - accuracy: 0.9778
Epoch 5/30
125/125 [==============================] - 13s 106ms/step - loss: 0.0596 - accuracy: 0.9818
Epoch 6/30
125/125 [==============================] - 18s 148ms/step - loss: 0.0408 - accuracy: 0.9870
Epoch 7/30
125/125 [==============================] - 14s 108ms/step - loss: 0.0366 - accuracy: 0.9893
Epoch 8/30
125/125 [==============================] - 13s 108ms/step - loss: 0.0295 - accuracy: 0.9900
Epoch 9/30
125/125 [==============================] - 13s 108ms/step - loss: 0.0231 - accuracy: 0.9935
Epoch 10/30
125/125 [==============================] - 19s 149ms/step - l

In [ ]:
model.save('/content/drive/MyDrive/data/손글씨/model.h5')

In [3]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/data/손글씨/model.h5')

In [4]:
test = pd.read_csv('/content/drive/MyDrive/data/손글씨/test/test_data.csv')
test.head()

,file_name
0,idx0001.png
1,idx0002.png
2,idx0003.png
3,idx0004.png
4,idx0005.png


In [5]:
test_images = []
counts = 0

for image_name in test['file_name']:
    image = np.array(Image.open('/content/drive/MyDrive/data/손글씨/test/' + image_name)).flatten()
    test_images.append(image)
    counts+=1
    if counts%500==0:
      print(counts)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [6]:
image_full_test = pd.DataFrame(test_images)

image_full_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
data_x = image_full_test/255.0
data_x = data_x.values.reshape(-1,28,28,1)

In [8]:
pred = model.predict(data_x)

In [11]:
predict = np.argmax(pred, axis = 1)

In [12]:
predict

array([1, 0, 8, ..., 3, 9, 0])

In [17]:
sub = pd.read_csv('/content/drive/MyDrive/data/손글씨/sample_submission.csv')
sub.head()

,file_name,label
0,idx0001.png,-1
1,idx0002.png,-1
2,idx0003.png,-1
3,idx0004.png,-1
4,idx0005.png,-1


In [18]:
sub['label'] = predict
sub.head()

,file_name,label
0,idx0001.png,1
1,idx0002.png,0
2,idx0003.png,8
3,idx0004.png,3
4,idx0005.png,5


In [21]:
sub.to_csv('/content/drive/MyDrive/data/손글씨/sub.csv', index = False)